In [50]:
#importing all files
import regex as re 
import glob
import unicodedata as ucd
import numpy as np
import pandas as pd
from collections import Counter
import json
import random

In [51]:
#putting movie files in a list
list_of_srts = list(glob.glob('*.srt'))
list_of_srts

['bumblebee.srt', 'Endgame.srt', 'shazam.srt', 'sherlock.srt', 'thor.srt']

In [52]:
#function to check if a sentence has hindi
def has_hindi(s):
    u_codes = [ucd.name(char).split(' ')[0] for char in s]
    is_dev = [code=='DEVANAGARI' for code in u_codes]
    return any(is_dev)

#function to make the sentences by parsing data
def make_sentence(line):
    sentence = ''
    try:
        if has_hindi(line):
            for i, char in enumerate(line):
                if char in allowed:
                    if char == ' ':
                        sentence += char
                    else:
                        sentence += ' ' + char
                elif ucd.name(char).split(' ')[0] == 'DEVANAGARI':
                    sentence += char
    except:
        pass
    return(sentence)

In [53]:
#adding valid sentences to dataframe
x = 0
data = []
total_hindi_lines = []
allowed = [',', ' ', '?', '!', '।']
for i in list_of_srts:
    list_lines = []
    hindi_sentences = []
    try:
        with open(i, encoding='utf8') as f:
            list_lines += f.read().splitlines()
            for line in list_lines:
                sentence = make_sentence(line)
                if len(sentence) > 0:
                    hindi_sentences.append(sentence.strip())
                    total_hindi_lines.append(sentence.strip())
    except:
        print(i)
    for sentence in hindi_sentences:
        words = sentence.split(' ')
        counterr = len(words)
        data.append(['Movies',list_of_srts[x].split('.')[0].capitalize(),sentence,counterr])
    x = x+1
print(len(data))

9179


In [54]:
#creating a reference dictionary
Dictionary = {'Source':'Name of the source from where the sentence is taken', 
              'Subsource':'Name of the subsource', 
              'Sentence':'The sentence from the subsource:='+str(len(total_hindi_lines)) ,
              'No. of words':'The number of words in the given sentence'}

In [55]:
#counting number of unique words
corpus = ' '.join(total_hindi_lines)
corpus = corpus.split(' ')
corpus = list(set(corpus))
corpus_set = set(corpus)
print(len(corpus))


#loading news channel words
np_array = np.load('/Users/jessjitsingh/Desktop/Jessjit/Internships/Bobble/unique_words_news.npy')
news_list = np_array.flatten()
news_set = set(news_list)

#concatenating the two unique word sets
final_unique_word_set = news_set | corpus_set
final_unique_word_list = list(final_unique_word_set)

#converting corpus to numpy and adding to dataframe
final_numpy_array = np.array(final_unique_word_list)
df_unique_words = pd.DataFrame(final_numpy_array, columns = ['Unique Words in Text'])
print(len(df_unique_words))

4373
62425


In [56]:
#loading Mobile data CSV File 
df_mobile_data = pd.read_csv (r'/Users/jessjitsingh/Desktop/Jessjit/Internships/Bobble/hindi_sentences_from_chat.csv')
print(len(df_mobile_data))

3462378


In [57]:
#cleaning,validating and adding mobile data
i = 0
ser = df_mobile_data['text']
frequency = df_mobile_data['frequency']
for f in frequency:
    if int(f)>30:
        sen = ser[i]
        integer = frequency[i]
        msentence = make_sentence(sen)
        if len(msentence)>0:
            words = msentence.split(" ")
            words = set(words)
            intersect = final_unique_word_set & words
            if(intersect==words):
                w = ser[i].split(" ")
                v = len(w)
                n = int(integer)
                l = ['Mobile Data', 'Personal Chat', msentence, v]
                data += [l]*n
        i = i+1
    else:
        break
print(len(data))

191974


In [58]:
#loading news file for new sentences
df_news_sentences = pd.read_csv (r'/Users/jessjitsingh/Desktop/Jessjit/Internships/Bobble/hindi_sentences_from_news.csv')
print(len(df_news_sentences))

348435


In [59]:
df_final_sentences = pd.DataFrame(data, columns = ['Source', 'Subsource', 'Sentence', 'No. of words'])
df_final_sentences = pd.concat([df_final_sentences, df_news_sentences], ignore_index=True)
print(len(df_final_sentences))

540409


In [60]:
#visualising length of the words
#df_final_sentences.plot.bar(x='Sentence',y='No. of words',rot=0)

In [61]:
def add_annotations(s,n):
    list1 = s.split()
    num1 = len(list1)
    if num1>2:
        x = random.randint(1,num1-2)
        if n==1:
            list1.insert(x,' <COUGH> ')
        if n==2:
            list1.insert(x,' <SNEEZE> ')
        if n==3:
            list1.insert(x,' <CLEAR-THROAT> ')
        if n==4:
            list1.insert(x,' <PAUSE-'+str(random.randint(1,4))+'-SECONDS> ')
        s = ' '.join(list1)
    return s
def annotate(data_sentence):
    x = random.randint(1,101)
    if x<30:
        y = random.randint(1,101)
        if y>50:
            data_sentence = add_annotations(data_sentence,4)
        else:
            z = random.randint(1,101)
            if z<33:
                data_sentence = add_annotations(data_sentence,1)
            elif 33<=z<66:
                data_sentence = add_annotations(data_sentence,2)
            else:
                data_sentence = add_annotations(data_sentence,3)
    return(data_sentence)

In [62]:
length = len(df_final_sentences)
df_final_sentences['Sentence'] = df_final_sentences['Sentence'].apply(annotate)
df_final_sentences = df_final_sentences.drop(['Unnamed: 0'],axis=1)
print(len(df_final_sentences))

540409


In [63]:
#exporting all data as Files
df_final_sentences.to_csv (r'/Users/jessjitsingh/Desktop/Jessjit/Internships/Bobble/ALL_SENTENCES_AND_SOURCES.csv', index = None, header=True)
df_unique_words.to_csv(r'/Users/jessjitsingh/Desktop/Jessjit/Internships/Bobble/ALL_WORDS.csv',index = None, header=True )
with open('Dictionary.txt', 'w') as file:
    file.write(json.dumps(Dictionary))
file.close()

In [64]:
df_only_final_sentences = df_final_sentences.drop(['Source','Subsource','No. of words'], axis=1)
print(len(df_only_final_sentences))
df_only_final_sentences.to_csv (r'/Users/jessjitsingh/Desktop/Jessjit/Internships/Bobble/ONLY_SENTENCES.csv', index = None, header=True)

540409
